In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

import re, string
import nltk

from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

from transformers import TFDistilBertModel
from transformers import DistilBertTokenizerFast

import tensorflow as tf
from tensorflow import keras


from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df= pd.read_csv('/kaggle/input/twitter-data/processed_twitter_data.csv')
print(df.head())

                                      processed_text  sentiment
0  USER URL aww that bummer you shoulda got david...          0
1  is upset that he can update his facebook by te...          0
2  USER dived many time for the ball managed to s...          0
3      my whole body feel itchy and like it on fire           0
4  USER no it not behaving at all mad why am here...          0


In [3]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['sentiment'], test_size=0.2, random_state=0)
print(f'Data Split into Training and Test sets done.')

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
print(f'Training set split into Training and Validation sets done.')

Data Split into Training and Test sets done.
Training set split into Training and Validation sets done.


In [5]:
MAX_LEN=100

In [6]:
def tokenize(data,max_len=MAX_LEN) :
    input_ids = []
    attention_masks = []
    for i in range(len(data)):
        encoded = tokenizer.encode_plus(
            data.iloc[i],
            add_special_tokens=True,
            max_length=MAX_LEN,
            padding='max_length',
            return_attention_mask=True
        )
        input_ids.append(encoded['input_ids'])
        attention_masks.append(encoded['attention_mask'])
    return np.array(input_ids),np.array(attention_masks)

In [7]:
train_input_ids, train_attention_masks = tokenize(X_train, MAX_LEN)
val_input_ids, val_attention_masks = tokenize(X_valid, MAX_LEN)
test_input_ids, test_attention_masks = tokenize(X_test, MAX_LEN)

In [8]:
distilbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
def create_model(bert_model, max_len=MAX_LEN):
    
    opt = tf.keras.optimizers.Adam(learning_rate=1e-5)
    loss = tf.keras.losses.BinaryCrossentropy()
    accuracy = tf.keras.metrics.BinaryAccuracy()


    input_ids = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    attention_masks = tf.keras.Input(shape=(max_len,),dtype='int32')
    
    embeddings = distilbert_model([input_ids,attention_masks])[0]
    
    output = tf.keras.layers.Dense(1, activation="sigmoid")(embeddings[:, 0, :])
    
    model = tf.keras.models.Model(inputs = [input_ids,attention_masks], outputs = output)
    
    model.compile(opt, loss=loss, metrics=accuracy)
    
    
    return model

In [10]:
model = create_model(distilbert_model, MAX_LEN)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 tf_distil_bert_model (TFDi  TFBaseModelOutput(last_hid   6636288   ['input_1[0][0]',             
 stilBertModel)              den_state=(None, 100, 768)   0          'input_2[0][0]']             
                             , hidden_states=None, atte                                           
                             ntions=None)                                                     

In [11]:
distillbert_mod = model.fit([train_input_ids,train_attention_masks], y_train, validation_data=([val_input_ids,val_attention_masks], y_valid), epochs=3, batch_size=64)

Epoch 1/3


I0000 00:00:1727501536.945716     110 service.cc:145] XLA service 0x7b7322cfc0f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727501536.945773     110 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1727501537.131265     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


16000/16000 [==============================] - 5921s 368ms/step - loss: 0.3757 - binary_accuracy: 0.8305 - val_loss: 0.3461 - val_binary_accuracy: 0.8466
Epoch 2/3
16000/16000 [==============================] - 5883s 368ms/step - loss: 0.3284 - binary_accuracy: 0.8560 - val_loss: 0.3401 - val_binary_accuracy: 0.8514
Epoch 3/3
16000/16000 [==============================] - 5878s 367ms/step - loss: 0.2974 - binary_accuracy: 0.8713 - val_loss: 0.3430 - val_binary_accuracy: 0.8520


In [12]:
result_distillbert = model.predict([test_input_ids,test_attention_masks])

10000/10000 [==============================] - 595s 59ms/step


In [13]:
print(result_distillbert)

[[0.33353427]
 [0.02378853]
 [0.3659775 ]
 ...
 [0.995893  ]
 [0.7602101 ]
 [0.845325  ]]


In [14]:
import pickle

with open('Sentiment-DistillBERT_3.pickle', 'wb') as file:
    pickle.dump(distillbert_mod, file)

/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [16]:
y_pred_distillbert = (result_distillbert > 0.5).astype(int)

In [17]:
y_pred_distillbert = y_pred_distillbert.flatten()

In [18]:
conf_matrix = confusion_matrix(y_test, y_pred_distillbert)
print('Confusion Matrix:\n', conf_matrix)

Confusion Matrix:
 [[139023  20792]
 [ 26170 134015]]


In [19]:
print('\tClassification Report for DistilBERT:\n\n', classification_report(y_test, y_pred_distillbert, target_names=['Negative', 'Positive']))

	Classification Report for DistilBERT:

               precision    recall  f1-score   support

    Negative       0.84      0.87      0.86    159815
    Positive       0.87      0.84      0.85    160185

    accuracy                           0.85    320000
   macro avg       0.85      0.85      0.85    320000
weighted avg       0.85      0.85      0.85    320000

